# Luxembourg sealing degree

In [8]:
## imp raster allready re-projected:
input_raster = r"N:\C2205_FAIRiCUBE\f02_data\d060_data_LUXEMBOURG\f01_Lux_neopyhtes\f02_luxembourg_data\f01_raw_data\land_cover\land_cover_2021_20cm_aoi.tif"
out_raster = r"N:\C2205_FAIRiCUBE\f02_data\d060_data_LUXEMBOURG\f01_Lux_neopyhtes\f02_luxembourg_data\f01_raw_data\land_cover\land_cover_2021_20cm_res.tif"


## reference poly: 10x10m:
reference_polygon = r"N:\C2205_FAIRiCUBE\f02_data\d060_data_LUXEMBOURG\f01_Lux_neopyhtes\f02_luxembourg_data\f00_fishnet\fishnet_10m_2169.shp"

## output:
out_10m_vector =r"N:\C2205_FAIRiCUBE\f02_data\d060_data_LUXEMBOURG\f01_Lux_neopyhtes\f02_luxembourg_data\f01_raw_data\land_cover\landcover_2021.shp"


## zonal statistics and calc. of normal. index

In [40]:
import rasterio
import numpy as np

# Define the reclassification rules
# This is a simple example where values are reclassified:
def reclassify(value):
    if value == 0:
        return 0
    elif 1 <= value <= 20:
        return 1
    elif 21 <= value <= 255:
        return 0
    else:
        return value  # No change for other values


input_file =  r"N:\C2205_FAIRiCUBE\f02_data\d060_data_LUXEMBOURG\f01_Lux_neopyhtes\f02_luxembourg_data\f01_raw_data\land_cover\land_cover_2021.tif"
output_file = r"N:\C2205_FAIRiCUBE\f02_data\d060_data_LUXEMBOURG\f01_Lux_neopyhtes\f02_luxembourg_data\f01_raw_data\land_cover\land_cover_2021_20cm_reclassify.tif"

# Open the raster file
with rasterio.open(input_file) as src:
    # Check the number of bands
    band_count = src.count
   
    print(f"Number of bands in the input raster: {band_count}")
    
    # Read the first band (assuming the raster has at least one band)
    if band_count >= 1:
        raster_data = src.read(1)  # Read the first band
        print(f"Shape of the raster data: {raster_data.shape}")
        
        # Apply the reclassification function to each pixel value
        reclassified_data = np.vectorize(reclassify)(raster_data)
        
        # Copy the metadata of the source file
        metadata = src.meta.copy()
        
        # Write the reclassified data to a new file
        with rasterio.open(output_file, 'w', **metadata) as dst:
            dst.write(reclassified_data, 1)  # Write to the first band

        print(f"Reclassified raster saved as {output_file}")
    else:
        print("The input raster has no bands.")

Number of bands in the input raster: 1
Shape of the raster data: (52200, 48580)


MemoryError: Unable to allocate 18.9 GiB for an array with shape (52200, 48580) and data type object

In [41]:
import rasterio
import numpy as np

# Define the reclassification rules
def reclassify(value, nodata):
    if value == nodata:
        return 0
    elif 0 <= value <= 100:
        return 1
    elif 101 <= value <= 200:
        return 2
    elif 201 <= value <= 255:
        return 3
    else:
        return value  # No change for other values

# File paths
input_file =  r"N:\C2205_FAIRiCUBE\f02_data\d060_data_LUXEMBOURG\f01_Lux_neopyhtes\f02_luxembourg_data\f01_raw_data\land_cover\land_cover_2021.tif"
output_file = r"N:\C2205_FAIRiCUBE\f02_data\d060_data_LUXEMBOURG\f01_Lux_neopyhtes\f02_luxembourg_data\f01_raw_data\land_cover\land_cover_2021_20cm_reclassify.tif"

# New nodata value
new_nodata_value = 0  # or whatever value you want to use as the new nodata value

# Open the raster file
with rasterio.open(input_file) as src:
    # Get the current nodata value
    current_nodata_value = src.nodata
    print(f"Current nodata value: {current_nodata_value}")
    
    # Copy the metadata and update the nodata value
    metadata = src.meta.copy()
    metadata.update(nodata=new_nodata_value)
    
    # Define the block size (adjust based on your memory capacity)
    block_size = 1024
    
    # Calculate the number of blocks in both dimensions
    num_blocks_x = (src.width + block_size - 1) // block_size
    num_blocks_y = (src.height + block_size - 1) // block_size
    
    # Create an output dataset
    with rasterio.open(output_file, 'w', **metadata) as dst:
        # Process each block
        for i in range(num_blocks_y):
            for j in range(num_blocks_x):
                # Define the window to read
                window = rasterio.windows.Window(
                    j * block_size, i * block_size,
                    min(block_size, src.width - j * block_size),
                    min(block_size, src.height - i * block_size)
                )
                
                # Read the data within the window
                raster_data = src.read(1, window=window)
                
                # Apply the reclassification function to each pixel value
                reclassified_data = np.vectorize(reclassify)(raster_data, current_nodata_value)
                
                # Write the reclassified data to the output dataset
                dst.write(reclassified_data, window=window, indexes=1)

print(f"Reclassified raster saved as {output_file}")


Current nodata value: 0.0
Reclassified raster saved as N:\C2205_FAIRiCUBE\f02_data\d060_data_LUXEMBOURG\f01_Lux_neopyhtes\f02_luxembourg_data\f01_raw_data\land_cover\land_cover_2021_20cm_reclassify.tif


In [42]:

import rasterio
import numpy as np
from rasterio.windows import Window


input_file =output_file
output_file = input_file[:-4]+'_10m.tif'

# Desired new resolution
new_resolution = 10

# Open the raster file
with rasterio.open(input_file) as src:
    # Read the original raster data
    raster_data = src.read(1)
    original_transform = src.transform
    original_resolution = src.res
    original_width = src.width
    original_height = src.height
    
    print(f"Original resolution: {original_resolution}")
    print(f"Original width: {original_width}, Original height: {original_height}")
    
    # Calculate the new width and height
    new_width = int(original_width * original_resolution[0] / new_resolution)
    new_height = int(original_height * original_resolution[1] / new_resolution)
    
    print(f"New width: {new_width}, New height: {new_height}")

    # Initialize the new raster data with zeros
    new_raster_data = np.zeros((new_height, new_width), dtype=raster_data.dtype)
    
    # Aggregate the data
    for i in range(new_height):
        for j in range(new_width):
            # Calculate the window to read
            window = Window(j * new_resolution // original_resolution[0],
                            i * new_resolution // original_resolution[1],
                            new_resolution // original_resolution[0],
                            new_resolution // original_resolution[1])
            window_data = src.read(1, window=window)
            
            # Sum the window data and assign to the new raster data
            new_raster_data[i, j] = np.sum(window_data)
    
    # Update the metadata
    new_transform = rasterio.transform.from_origin(
        original_transform.c,
        original_transform.f,
        new_resolution,
        new_resolution
    )
    new_meta = src.meta.copy()
    new_meta.update({
        "height": new_height,
        "width": new_width,
        "transform": new_transform
    })
    
    # Write the new raster data to a file
    with rasterio.open(output_file, 'w', **new_meta) as dst:
        dst.write(new_raster_data, 1)

print(f"Aggregated raster saved as {output_file}")


Original resolution: (0.2000000000000003, 0.2)
Original width: 48580, Original height: 52200
New width: 971, New height: 1044
Aggregated raster saved as N:\C2205_FAIRiCUBE\f02_data\d060_data_LUXEMBOURG\f01_Lux_neopyhtes\f02_luxembourg_data\f01_raw_data\land_cover\land_cover_2021_20cm_reclassify_10m.tif


## 